In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.metrics import precision_recall_curve, roc_curve, auc

In [2]:
# Command to use the full widht of the screen
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }/style>"))

In [7]:
def BACN(init_nodes, final_nodes, m_param, l_param=0, CN=False, DIR=False):
    assert init_nodes > 1
    assert init_nodes < final_nodes
    assert init_nodes >= m_param
    assert l_param >= 0
    start = time.time()
    A = np.zeros((final_nodes, final_nodes)) 
    for i in range(init_nodes):
        for j in range(i+1, init_nodes):
            A[i,j] = 1
            A[j,i] = 1
    for i in range(init_nodes, final_nodes):
        if DIR:
            p_deg_target = np.sum(A[:i,:], axis=1)
            p_deg_target /= np.sum(p_deg_target)
            p_deg_source = np.sum(A[:,:i], axis=0)
            p_deg_source /= np.sum(p_deg_source)
            rand_target = np.random.choice(i, size=m_param, p=p_deg_target, replace=False)
            A[rand_target, i] = 1
            rand_source = np.random.choice(i, size=m_param, p=p_deg_source, replace=False)
            A[i, rand_source] = 1
        else:
            p_deg = np.sum(A, axis=1)[:i]
            p_deg /= np.sum(p_deg)
            rand_node = np.random.choice(i, size=m_param, p=p_deg, replace=False)
            A[i, rand_node] = A[rand_node, i] = 1
        print("\nA")
        print(A)
        if CN and l_param > 0:
            S = A[:i+1,:i+1]
            print("First S")
            print(S)
            cn_mat = S @ S
            print("cn_mat")
            print(cn_mat)
            S[np.eye(S.shape[0], dtype=bool)] = -1
            print("Second S")
            print(S)
            missing = np.nonzero(S==0)
            print("missing:", missing, "and missing[0].size/2:", missing[0].size/2)
            if missing[0].size/2 >= l_param:
                print("FIRST if")
                p_cn = cn_mat[missing]
                p_cn /= np.sum(p_cn)
                print("p_cn", p_cn)
                if np.isnan(p_cn).all():
                    print("SECOND if")
                    rand_edge = np.random.choice(missing[0].size, size=l_param)
                    print("rand_edge", rand_edge)
                else:
                    print("SECOND else")
                    rand_edge = np.random.choice(missing[0].size, size=l_param, p=p_cn)
                    print("rand_edge", rand_edge)
                a = missing[0][rand_edge]
                print("a", a)
                b = missing[1][rand_edge]
                print("b", b)
                A[a,b] = 1
                print("A")
                print(A)
                S[np.eye(S.shape[0], dtype=bool)] = 0
                print("Third S")
                print(S)
            S[np.eye(S.shape[0], dtype=bool)] = 0
            print("Fourth S")
            print(S)
    end = time.time()
    #print('Elapsed time:', end - start, 'seconds')
    return A

In [8]:
final_nodes = 5
init_nodes = 3
AA = np.zeros((final_nodes, final_nodes)) 
for i in range(init_nodes):
    for j in range(i+1, init_nodes):
        AA[i,j] = 1
        AA[j,i] = 1
AA

array([[0., 1., 1., 0., 0.],
       [1., 0., 1., 0., 0.],
       [1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [13]:
testBACN = BACN(3,6,1,2,CN=True)#,DIR=True)
testBACN


A
[[0. 1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
First S
[[0. 1. 1. 0.]
 [1. 0. 1. 0.]
 [1. 1. 0. 1.]
 [0. 0. 1. 0.]]
cn_mat
[[2. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 3. 0.]
 [1. 1. 0. 1.]]
Second S
[[-1.  1.  1.  0.]
 [ 1. -1.  1.  0.]
 [ 1.  1. -1.  1.]
 [ 0.  0.  1. -1.]]
missing: (array([0, 1, 3, 3]), array([3, 3, 0, 1])) and missing[0].size/2: 2.0
FIRST if
p_cn [0.25 0.25 0.25 0.25]
SECOND else
rand_edge [3 2]
a [3 3]
b [1 0]
A
[[-1.  1.  1.  0.  0.  0.]
 [ 1. -1.  1.  0.  0.  0.]
 [ 1.  1. -1.  1.  0.  0.]
 [ 1.  1.  1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
Third S
[[0. 1. 1. 0.]
 [1. 0. 1. 0.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]]
Fourth S
[[0. 1. 1. 0.]
 [1. 0. 1. 0.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]]

A
[[0. 1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0.]
 [1. 1. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
First S
[[0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [1. 1. 0.

array([[0., 1., 1., 1., 0., 0.],
       [1., 0., 1., 0., 1., 0.],
       [1., 1., 0., 1., 1., 0.],
       [1., 1., 1., 0., 1., 1.],
       [1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [6]:
def cns_number(x, links):
    s = links.shape[0]
    cn = np.zeros(s)
    neighbors = {i:np.argwhere(x[i]) for i in range(x.shape[0])}
    for i in range(s):
        inter = np.intersect1d(neighbors[links[i,0]], neighbors[links[i,1]])
        cn[i] = inter.size
    return cn

In [ ]:
cns_number(testBACN, np.array([[0,0], [0,1], [0,2], [0,3], [0,4], [0,5]]))

In [ ]:
def get_links_to_del(x, fraction=0.1, loops=1):
    present = np.argwhere(np.triu(x, 1) == 1)
    num_to_del = int(present.shape[0] * fraction)
    idx_to_del = [np.random.choice(present.shape[0], size=num_to_del, replace=False) for i in range(loops)]
    links_to_del = np.array([present[idx] for idx in idx_to_del])
    return links_to_del

In [ ]:
def delete_links(x, links_to_del):
    """Deletes the specified links from an adjacency matrix."""
    z = np.copy(x)
    for link in links_to_del:
        z[link[0], link[1]] = 0
    return z

In [ ]:
def evaluate_predictions(true, pred, plot=False):
    precision, recall, _ = precision_recall_curve(true, pred)
    fpr, tpr, _ = roc_curve(true, pred)
    aupr = auc(recall, precision)
    auroc = auc(fpr, tpr)
    if plot:
        fig, ax = plt.subplots()
        ax.plot(recall, precision, 'go', label='Precision-Recall curve')
        ax.plot(fpr, tpr, '.', label='ROC curve')
        plt.legend()
        plt.show()
    return aupr, auroc

In [ ]:
def cross_val_from_notebook(adjacency, links_to_del, loops=1, raw_output=False, verbose=False, plot=False):
    if raw_output:
        scores_list = []
        true_list = []
    else:
        results = np.zeros((loops, 2))
    for i in range(loops):
        if verbose:
            print('Trial {} of {}'.format(i+1, loops))
        x_ = delete_links(adjacency, links_to_del[i])
        missing = np.argwhere(x_ == 0)
        not_false = adjacency[missing[:,0], missing[:,1]]
        scores = cns_number(x_, missing)
        if raw_output:
            scores_list.append(scores)
            true_list.append(not_false)
        else:
            results[i] = evaluate_predictions(not_false, scores, plot=plot)
    if raw_output:
        return np.array(true_list).flatten(), np.asarray(scores_list).flatten()
    else:
        return results

In [ ]:
def running_all_parameters(init_nodes, final_nodes, l, loops=10, CN=False):
    fraction = 0.1
    raw_output=False
    verbose = False
    plot = False
    areas_under_curves = np.zeros((init_nodes+1, 2))
    for m in range(init_nodes+1):
        start = time.time()
        B = BACN(init_nodes, final_nodes, m, l, CN)
        links_to_del = get_links_to_del(B, fraction=fraction, loops=loops)
        r1 = cross_val_from_notebook(B, links_to_del)#, loops=loops, raw_output=raw_output, verbose=verbose, plot=plot)
        areas_under_curves[m,:] = np.mean(r1, axis=0)
        end = time.time()
    return areas_under_curves

In [ ]:
def ploting_all_parameters(final, lmax, loops, CN):
    for i in range(2, final):
        start = time.time()
        plt.figure(figsize=(20,3))
        for j in range(1, lmax+1):
            r_p = running_all_parameters(i, final, j-1, loops, CN)
            plt.subplot(1,lmax,j)
            plt.title("Initial:{}, final:{}\nCN-parm:{}".format(i, final, j))
            plt.xlabel("m parameter")
            plt.plot(r_p[:,0], 'bx--', label='AUPR')
            plt.plot(r_p[:,1], 'ro--', label='AUROC')
            plt.legend()
        plt.show()
        end = time.time()

In [ ]:
ploting_all_parameters(final=10, lmax=6, loops=1, CN=True)